# Tutorial

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1RCdAyrodMFn8CcjI4iiWaDlIGyH3f6cD?usp=sharing)

[![View Source on GitHub](https://img.shields.io/badge/github-view%20source-black.svg)](https://github.com/yuliya1324/ru_attacker/blob/main/Tutorial.ipynb)

## Installation

In [ ]:
!pip install ru_attacker

In [ ]:
import nltk
nltk.download('punkt')

## Usage example

Set model

In [5]:
from ru_attacker.models import RobertaModel
roberta_checkpoints = "Roberta_checkpoints"
ruRoberta = RobertaModel(roberta_checkpoints)

Set dataset

In [6]:
from ru_attacker.models import get_data
data_dir = "TERRa/val.jsonl" 
data = get_data(data_dir)

Set attack

You have to define `transformation`, `goal_function` and `type_perturbation`. `constraints` and `search_method` are optional


In [7]:
from ru_attacker.attacks.transformations import BackTranslation  # transformation
from ru_attacker.attacks.goal_function import LabelPreserving  # goal function
from ru_attacker.attacks.constraints import GrammarAcceptability, SemanticSimilarity  # constraints
from ru_attacker.attacks.search_method import GreedySearch  # search method
from ru_attacker.attacks import Attack  # attack wrapper
backtranslation = Attack(
        transformation=BackTranslation(languages=["en", "fr", "de"]),  # you can set languages manually or use the default ones
        goal_function=LabelPreserving(),
        type_perturbation="hypothesis",  # to what part perturbation is applied {"hypothesis", "premise"}
        constraints=[GrammarAcceptability(), SemanticSimilarity()],
        search_method=GreedySearch()
    )

Downloading:   0%|          | 0.00/908 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.80G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.54M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/272 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Unzipping /tmp/tmpcfp5xwk8.zip to /root/.cache/language_tool_python.
Downloaded https://www.languagetool.org/download/LanguageTool-5.7.zip to /root/.cache/language_tool_python.


Attack model and view results

In [8]:
results = backtranslation.attack(ruRoberta, data)


                  [Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:
                  entailment --> entailment
                  original premise: """Решение носит символический характер, так как взыскать компенсацию практически невозможно"", - отмечается в сообщении."
                  original hypothesis: Взыскать компенсацию не получится.

                  transformed: Компенсации не будет получено.

                  

                  [Succeeded / Failed / Skipped / Total] 1 / 1 / 0 / 2:
                  entailment --> not_entailment
                  original premise: Об этом вечером во вторник, 17 января, сообщила пресс-служба Спасательного департамента, отметив, что немецкую противотанковую мину Tellermine 42 обнаружили в на улице Кеэвисе в ходе земляных работ. Спасатели эвакуировали жителей окрестных домов, офисов и складских помещений. Уничтожать мину на месте не стали, поскольку это угрожало повреждению трассы трубопровода.
                  original hypothesis: На 

Convert results to DataFrame

In [9]:
import pandas as pd
dataframe = pd.DataFrame(results)
dataframe.head()

,original label,attacked label,original premise,original hypothesis,transformed,attack
0,1,1.0,"""""""Решение носит символический характер, так к...",Взыскать компенсацию не получится.,Компенсации не будет получено.,failed
1,1,0.0,"Об этом вечером во вторник, 17 января, сообщил...",На улице Кеэвисе жителей эвакуировали из-за мины.,На улице Кевиза люди были эвакуированы из шахт.,succeeded
2,1,0.0,"""В суде Центрального района Сочи завершилось р...",81-летний пенсионер застрелил обидчика дочери.,81-летний пенсионер убил свою дочь.,succeeded
3,1,1.0,"""Кто-то нарисовал на снегу контур человеческог...",Кто-то изобразил на снегу контур человеческого...,Иногда на снегу изображается контр человеческо...,failed
4,1,0.0,"Также установлено, что работники учреждения не...",В детском саду не все вовремя проходят медосмотр.,Детский сад не проводит медицинского осмотра с...,succeeded


## Create your own Attack or Model

Now lets create an attack that changes all character `ч` to `4` and `о` to `0`. For example, `Второй тайм резко отличался от первого.` --> `Вт0р0й
тайм резк0 0тли4ался 0т перв0г0.` We will create a new transformation class inherited from the `BasicTransformation` class

In [11]:
from ru_attacker.attacks.transformations import BasicTransformation
help(BasicTransformation)

Help on class BasicTransformation in module ru_attacker.attacks.transformations.basic_transformation:

class BasicTransformation(abc.ABC)
 |  A basic class for transformations
 |  
 |  Method resolution order:
 |      BasicTransformation
 |      abc.ABC
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  transform(self, text)
 |      a method to perform perturbation
 |      :param text: text to transform
 |      :return: list of transformed texts
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  __abstractmethods__ = frozenset({'transform'})



In [12]:
class CharTransformation(BasicTransformation):
    def __init__(self):
        self.di = {"ч": "4", "о": "0"}

    def transform(self, text):
        return ["".join([self.di[ch] if ch in self.di else ch for ch in text])]

`goal_function` is label-preserving, and the perturbation is applied to hypothesis. We will not set any constraints and search method here

In [13]:
char_attack = Attack(
    transformation=CharTransformation(),
    goal_function=LabelPreserving(),
    type_perturbation="hypothesis",
    constraints=None,
    search_method=None
)

In [14]:
results = char_attack.attack(ruRoberta, data)


                  [Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:
                  entailment --> entailment
                  original premise: """Решение носит символический характер, так как взыскать компенсацию практически невозможно"", - отмечается в сообщении."
                  original hypothesis: Взыскать компенсацию не получится.

                  transformed: Взыскать к0мпенсацию не п0лу4ится.

                  

                  [Succeeded / Failed / Skipped / Total] 0 / 2 / 0 / 2:
                  entailment --> entailment
                  original premise: Об этом вечером во вторник, 17 января, сообщила пресс-служба Спасательного департамента, отметив, что немецкую противотанковую мину Tellermine 42 обнаружили в на улице Кеэвисе в ходе земляных работ. Спасатели эвакуировали жителей окрестных домов, офисов и складских помещений. Уничтожать мину на месте не стали, поскольку это угрожало повреждению трассы трубопровода.
                  original hypothesis: На 

Now let's look at this transformations

In [15]:
dataframe = pd.DataFrame(results)
dataframe.head()

,original label,attacked label,original premise,original hypothesis,transformed,attack
0,1,1,"""""""Решение носит символический характер, так к...",Взыскать компенсацию не получится.,Взыскать к0мпенсацию не п0лу4ится.,failed
1,1,1,"Об этом вечером во вторник, 17 января, сообщил...",На улице Кеэвисе жителей эвакуировали из-за мины.,На улице Кеэвисе жителей эвакуир0вали из-за мины.,failed
2,1,1,"""В суде Центрального района Сочи завершилось р...",81-летний пенсионер застрелил обидчика дочери.,81-летний пенси0нер застрелил 0бид4ика д04ери.,failed
3,1,1,"""Кто-то нарисовал на снегу контур человеческог...",Кто-то изобразил на снегу контур человеческого...,Кт0-т0 из0бразил на снегу к0нтур 4ел0ве4еск0г0...,failed
4,1,1,"Также установлено, что работники учреждения не...",В детском саду не все вовремя проходят медосмотр.,В детск0м саду не все в0время пр0х0дят мед0см0тр.,failed


Feel free to attack your own model, using `BasicModel` or any of implemented classes, or construct your own attacks with the help of `BasicTransformation`, `BasicSearchMethod`, `BasicGoalFunction` and `BasicConstraint`!